Trabajo de curso

Importamos los paquetes y librerías necesarias.

In [119]:
import cv2
import numpy as np
from cvzone.HandTrackingModule import HandDetector
import os, math
import time
from win32api import GetSystemMetrics
import pyautogui


Definimos las variables globales que usaremos para controlar el trazado de las líneas.

In [120]:
# Detector de manos de CVZone
detector = HandDetector(detectionCon=0.8, maxHands=1)

# Variables para dibujar
drawing = False  # Si está dibujando o no
points_dict = {}  # Diccionario para almacenar los puntos asociados a colores, grosores y modos de pintura

# Lista de colores y grosores disponibles
colors = [(0,0,0), (0, 0, 255), (0, 255, 0), (255, 0, 0), (255, 255, 0), (255, 0, 255), (0, 255, 255), (255,255,255)]  # Rojo, verde, azul, amarillo, magenta, cian
thicknesses = [5, 10, 15, 20]  # Lista de grosores disponibles
paint_modes = ['normal', 'highlighter', 'borde']  # Modos de pintura
images = ["boli", "marker", "borde"] #imagenes de los tipos de pintura

selection_delay = 1   # Tiempo en segundos de no dibujo tras seleccionar alguna opción en un menú
has_delay_time_passed = True  # Flag para saber si se ha seleccionado un color tras el menú de selección

# Opciones de pintura por defecto
current_color = colors[1]
current_thickness = thicknesses[1]
current_paint_mode = paint_modes[0]

# Lista que guarda en orden los trazos dibujados
drawings_over_time = []
undo_dict = {}
redo_history = []

# Logica para borrado
erasing = False # Si está borrando o no
last_action = None

# Temporizador para el gesto "thumbs up" (captura de pantalla del dibujo)
thumbs_up_start_time = None
time_to_capture = 1.5  # Tiempo en segundos para mantener el gesto "thumbs up" y capturar la pantalla

# Diccionario para el primer color, grosor y modo de pintura, cuya lista contiene los puntos dibujados con esos valores
if (current_color, current_thickness, current_paint_mode) not in points_dict:
    points_dict[(current_color, current_thickness, current_paint_mode)] = []

# Flags para usar con las interfaces de selección de color, grosor y tipo de pintura
color_interface_active = False
thickness_interface_active = False
paint_mode_interface_active = False
undo_mode_active = False
allow_undo = True
redo_mode_active = False
allow_redo = True

Definimos las funciones que usaremos en el main

In [121]:
def distanceFunc(p1, p2):
    return math.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2)

# Función para interpolar los puntos y dibujar líneas continuas
def interpolate_points(p1, p2):
    points = []
    x1, y1 = p1
    x2, y2 = p2
    distance = int(distanceFunc(p1,p2))
    if distance == 0:
        return [p1]
    for i in range(1, distance + 1):
        t = i / distance
        x = int(x1 + (x2 - x1) * t)
        y = int(y1 + (y2 - y1) * t)
        if (x, y) not in points:
            points.append((x, y))
    return points

# Funciones auxiliares

def is_point_on_line(x, y, m, b, tolerance=50):

    """
    Verifica si el punto (x, y) está aproximadamente sobre la recta y = mx + b
    con una tolerancia de error (tolerance).
    """
    y_expected = m * x + b
    return abs(y - y_expected) <= tolerance

# Lógica para saber cual fue la última en ponerse a true
def update_actions():
    global drawing, erasing, last_action
    if drawing and erasing:
        # Ambas son True en la misma iteración
        last_action = None
    elif drawing:
        last_action = 'drawing'
    elif erasing:
        last_action = 'erasing'

    # Si ninguna es True, mantiene el último valor
    return last_action

def detectar_gestos(hand, lmList):
    fingers = detector.fingersUp(hand)  # Estado de los dedos (1 = arriba, 0 = abajo)
    gordobx, gordoby, gordobz = lmList[1]
    # Gestos
    if fingers == [0, 0, 0, 0, 0]:
        return "Puño cerrado"
    elif fingers == [1, 1, 0, 0, 0]:
        return "L"
    elif fingers == [1, 1, 1, 1, 1]:
        for i in range(5):
            basex, basey, basez = lmList[i * 4 + 1]
            puntax,puntay,puntaz = lmList[i * 4 + 4]
            if abs(gordobz-basez)>=70:
                return False
            if abs(basez-puntaz)>=70:
                return False

            # Calcular la pendiente (m) y el valor b de la recta que pasa por la base y la punta
            if (puntax - basex) == 0:  # Evitar división por 0
                m = np.inf
            else:
                m = (puntay - basey) / (puntax - basex)
            
            b = basey - m * basex
            mediox,medioy,_ = lmList[i * 4 + 3]
            if not is_point_on_line(mediox, medioy, m, b):
                return False  # Si algún punto no está en la recta, devolver False

        return "Mano extendida"
    
    elif fingers == [0, 1, 1, 0, 0]:
        return "Señal de paz"
    
    elif fingers == [0, 1, 1, 1, 0]:
        three_fingers = (
            lmList[5][1] > lmList[6][1] > lmList[7][1] > lmList[8][1] and
            lmList[9][1] > lmList[10][1] > lmList[11][1] > lmList[12][1] and
            lmList[13][1] > lmList[14][1] > lmList[15][1] > lmList[16][1]
        )
        if three_fingers and distanceFunc((lmList[4][0], lmList[4][1]), (lmList[20][0], lmList[20][1])) < 50:
            return "Tres dedos arriba"
        
    elif fingers == [1, 0, 0, 0, 1]:
        return "Llamada"
    elif fingers == [0, 1, 0, 0, 0]:
        return "Dedo íncide arriba"
    return "Sin gesto"


def detect_color_selection(x, y, index_thumb_distance, color_rectangles, current_color, color_interface_active):
    if index_thumb_distance >= 50:
        return current_color, color_interface_active
    
    for color, (x1, y1, w, h) in color_rectangles:
        # Verificar si el clic está dentro del cuadrado
        if x1 <= x <= x1 + w and y1 <= y <= y1 + h:
            color_interface_active = False
            current_color = color
            break

    return current_color, color_interface_active

def detect_thickness_selection(x, y, index_thumb_distance, thickness_circles, current_thickness, thickness_interface_active):
    if index_thumb_distance >= 50:
        return current_thickness, thickness_interface_active
    
    for thickness, (center_x, center_y, radius) in thickness_circles:
        # Verificar si el clic está dentro del círculo de grosor
        if (x - center_x) ** 2 + (y - center_y) ** 2 <= radius ** 2:
            thickness_interface_active = False
            current_thickness = thickness
            break

    return current_thickness, thickness_interface_active     

def detect_paint_mode_selection(x, y, index_thumb_distance, mode_rectangles, current_paint_mode, paint_mode_interface_active):
    if index_thumb_distance >= 50:
        return current_paint_mode, paint_mode_interface_active
    
    for mode, (x1, y1, w, h) in mode_rectangles:
        # Verificar si el clic está dentro del rectángulo de modo de pintura
        if x1 <= x <= x1 + w and y1 <= y <= y1 + h:
            current_paint_mode = mode
            paint_mode_interface_active = False
            break

    return current_paint_mode, paint_mode_interface_active

def draw_color_selection_interface(img):
    color_rectangles = []
    num_colors = len(colors)
    rect_width = 60
    rect_height = 50
    spacing = 20
    start_x = (img.shape[1] - (num_colors * rect_width + (num_colors - 1) * spacing)) // 2
    start_y = img.shape[0] - 100

    # Dibujar el fondo semitransparente para toda la interfaz
    overlay = img.copy()
    background_x1 = start_x - 20
    background_y1 = start_y - 20
    background_x2 = start_x + num_colors * (rect_width + spacing) - spacing + 20
    background_y2 = start_y + rect_height + 20
    cv2.rectangle(overlay, (background_x1, background_y1), (background_x2, background_y2), (0, 0, 0), -1)
    alpha = 0.6  # Nivel de transparencia (0: completamente transparente, 1: completamente opaco)
    cv2.addWeighted(overlay, alpha, img, 1 - alpha, 0, img)

    # Dibujar los rectángulos individuales para cada color
    for i, color in enumerate(colors):
        x1, y1 = start_x + i * (rect_width + spacing), start_y
        cv2.rectangle(img, (x1, y1), (x1 + rect_width, y1 + rect_height), color, -1)
        color_rectangles.append((color, (x1, y1, rect_width, rect_height)))
    
    return color_rectangles

def draw_thickness_selection_interface(img, color, background=(0, 0, 0)):
    thickness_circles = []
    num_thicknesses = len(thicknesses)
    spacing = 20  # Espacio entre los círculos
    padding = 20  # Espaciado interno en el rectángulo
    circle_diameter = max(thicknesses) * 2
    rect_width = num_thicknesses * (circle_diameter + spacing) - spacing + padding * 2
    rect_height = circle_diameter + padding * 2
    start_x = (img.shape[1] - rect_width) // 2
    start_y = img.shape[0] - 200

    # Dibujar el fondo semitransparente
    overlay = img.copy()
    cv2.rectangle(overlay, (start_x, start_y), (start_x + rect_width, start_y + rect_height), background, -1)
    alpha = 0.6  # Nivel de transparencia (0: completamente transparente, 1: completamente opaco)
    cv2.addWeighted(overlay, alpha, img, 1 - alpha, 0, img)

    # Dibujar los círculos de grosor
    for i, thickness in enumerate(thicknesses):
        center_x = start_x + padding + i * (circle_diameter + spacing) + circle_diameter // 2
        center_y = start_y + rect_height // 2
        # Dibujar un círculo con borde negro y relleno con el color actual
        cv2.circle(img, (center_x, center_y), thickness, (0, 0, 0), -1)  # Borde negro
        cv2.circle(img, (center_x, center_y), thickness - 1, color, -1)  # Relleno con el color actual
        thickness_circles.append((thickness, (center_x, center_y, thickness)))

    return thickness_circles

def draw_paint_mode_interface(img):
    mode_rectangles = []
    num_modes = len(paint_modes)
    rect_width = 60
    spacing = 20

    # Ajustar la posición vertical (más abajo)
    start_x = (img.shape[1] - (num_modes * rect_width + (num_modes - 1) * spacing)) // 2
    start_y = img.shape[0] - 100  # Aquí puedes ajustar el valor para mover los rectángulos más abajo

    for i, mode in enumerate(paint_modes):
        x1, y1 = start_x + i * (rect_width + spacing), start_y
        
        # Crear el rectángulo semitransparente
        overlay = img.copy()
        alpha = 0.4  # Nivel de transparencia
        cv2.rectangle(overlay, (x1, y1), (x1 + rect_width, y1 + 50), (200, 200, 200), -1)
        cv2.addWeighted(overlay, alpha, img, 1 - alpha, 0, img)  # Fusionar el rectángulo semitransparente con la imagen
        
        # Cargar la imagen para el modo de pintura
        image_path = f'utils/'+images[i]+'.png'  # Ruta de imagenes
        mode_image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)  # Cargar con transparencia
        
        mode_image_resized = cv2.resize(mode_image, (rect_width, 50))

        # Separar los canales
        if mode_image_resized.shape[2] == 4:  # Si la imagen tiene 4 canales (RGBA)
            bgr_image = mode_image_resized[:, :, :3]  # Extraer los tres canales BGR
            alpha_channel = mode_image_resized[:, :, 3]  # Extraer el canal alfa
        else:
            bgr_image = mode_image_resized
            alpha_channel = None

        # Crear una máscara con el canal alfa si existe
        if alpha_channel is not None:
            # Crear una máscara donde el valor alfa es mayor que 0
            mask = alpha_channel / 255.0  # Normalizar el canal alfa entre 0 y 1

            # Superponer la imagen con el fondo (con transparencia)
            for c in range(0, 3):  # Para cada canal RGB
                img[y1:y1 + 50, x1:x1 + rect_width, c] = (
                    (1.0 - mask) * img[y1:y1 + 50, x1:x1 + rect_width, c] +
                    mask * bgr_image[:, :, c]
                )
        else:
            # Si no hay canal alfa, simplemente reemplaza la región
            img[y1:y1 + 50, x1:x1 + rect_width] = bgr_image

        # Agregar texto (opcional)
        #cv2.putText(img, f"{paint_modes[i]}", (x1 + 10, y1 + 35), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

        # Almacenar los rectángulos para detección de clics
        mode_rectangles.append((mode, (x1, y1, rect_width, 50)))
    
    return mode_rectangles

def draw_paint_mode(img, points, mode, thickness, color):
    if mode == 'normal':
        for point in points:
            if point is None:
                continue
            cv2.circle(img, point, thickness, color, -1)

    elif mode == 'highlighter':
        overlay = img.copy()
        bright_color = tuple(min(255, int(c * 5.0)) for c in color)  # Hacer el color más brillante
        for point in points:
            if point is None:
                continue
            cv2.circle(overlay, point, thickness , bright_color, -1)
        alpha = 0.6  # Transparencia para subrayador
        cv2.addWeighted(overlay, alpha, img, 1 - alpha, 0, img)

    elif mode == 'borde':
        #points = points_dict[(color, thickness, mode)]
        # Lista para almacenar las sublistas
        sub_lists = []
        current_list = []

        # Recorremos todos los puntos
        for point in points:
            if point is None:
                if current_list:  # Si hay una lista no vacía, la agregamos a sub_lists
                    sub_lists.append(current_list)
                current_list = []  # Reiniciamos la lista para los siguientes puntos
            else:
                current_list.append(point)

        # Agregar la última lista si no está vacía
        if current_list:
            sub_lists.append(current_list)

        for lista in sub_lists:
            # Convertir los puntos a un formato adecuado para polylines (array de numpy)
            points_array = np.array(lista, np.int32)
            points_array = points_array.reshape((-1, 1, 2))
            

            # Dibujar el borde (una línea gruesa alrededor de la línea)
            cv2.polylines(img, [points_array], isClosed=False, color=(255,255,255), thickness=thickness*2+3)

            # Dibujar la línea principal con el grosor especificado
            cv2.polylines(img, [points_array], isClosed=False, color=color, thickness=thickness*2)


def detect_thumbs_up(lmList, hands):
    thumb_tip = lmList[4]
    wrist = lmList[0]
    pinky_tip = lmList[20]
    pinky_mcp = lmList[17]
    # Mano detectada
    hand = hands[0]

    # Calcular la distancia entre la punta del meñique y su nudillo
    pinky_distance = math.sqrt((pinky_tip[0] - pinky_mcp[0])**2 + (pinky_tip[1] - pinky_mcp[1])**2)

    # Verificar si la distancia es menor a 30 px, ya que cuando se hace el gesto, ambos puntos son cercanos
    if pinky_distance > 25:
        return False

    # Calcular el ángulo entre la línea thumb_tip-wrist y el eje X
    delta_x = thumb_tip[0] - wrist[0]
    delta_y = thumb_tip[1] - wrist[1]
    angle = math.degrees(math.atan2(delta_y, delta_x))

    # Ajustar el ángulo para que esté en el rango [0, 180]
    if angle < 0:
        angle += 180

    # Verificar si el ángulo está entre 40 y 90 grados para ambas manos
    if (hand['type'] == 'Left' and not (40 <= angle <= 90)) or (hand['type'] == 'Right' and not (90 <= angle <= 140)):
        return False
    
    return True

def flash_effect(frame, duration=0.2, flash_color=(255, 255, 255)):
    """
    Aplica un efecto de flash al feed de video en tiempo real.
    """
    flash_frame = np.full_like(frame, flash_color, dtype=np.uint8)  # Frame completamente blanco
    start_time = time.time()
    while time.time() - start_time < duration:
        cv2.imshow("Screen", flash_frame)
        cv2.waitKey(1)

def border_effect(img, duration=0.5, thickness=20, color=(0, 255, 0)):
    """
    Dibuja un borde animado alrededor de la pantalla.
    """
    start_time = time.time()
    while time.time() - start_time < duration:
        bordered_img = img.copy()
        h, w, _ = img.shape
        cv2.rectangle(bordered_img, (thickness, thickness), (w - thickness, h - thickness), color, thickness)
        cv2.imshow("Screen", bordered_img)
        cv2.waitKey(50)

def replace_frames_with_effect(cap , duration=0.5, color=(255, 255, 255)):
    """
    Reemplaza los frames del feed con un efecto durante un período definido.
    """
    start_time = time.time()
    
    while time.time() - start_time < duration:
        ret, frame = cap.read()
        if not ret:
            break

        effect_frame = np.full_like(frame, color, dtype=np.uint8)

        # Muestra el frame con el efecto
        cv2.imshow("Camera", effect_frame)

        # Permite salir con 'q' durante el efecto
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

def undo():
    global drawings_over_time, points_dict, undo_dict, redo_history
    print("Points_dict sin mutar: ", points_dict)
    if drawings_over_time:
        drawing_type = drawings_over_time[-1]  # Obtener el último tipo de dibujo
        if drawing_type in points_dict:  # Verificar si el tipo existe en points_dict
            if drawing_type not in undo_dict:
                undo_dict[drawing_type] = []

            # Recorrer hacia atrás y extraer puntos hasta encontrar None
            while points_dict[drawing_type]:
                point = points_dict[drawing_type].pop()
                # Insertamos por delante
                undo_dict[drawing_type].append(point)
                if point is None:
                    break
        
        


        
        print("Points_dict undo: ", points_dict)
        redo_history.append(drawings_over_time.pop()) # Eliminar el último tipo de dibujo
        print("undo_dict: ", undo_dict)
        print("redo_history: ", redo_history)

def redo(redo_history, undo_dict):
    global drawings_over_time, points_dict
    if redo_history:
        drawing_type = redo_history.pop()  # Obtener el último tipo de dibujo
        if drawing_type in undo_dict:
            drawings_over_time.append(drawing_type)  # Agregar el tipo de dibujo eliminado a la lista de dibujos
            # Lista auxiliar para encontrar todos los puntos hasta None
            points_till_none = []
            while undo_dict[drawing_type]:
                point = undo_dict[drawing_type].pop()
                points_till_none.append(point)
                if point == None:
                    break
            points_dict[drawing_type].extend(reversed(points_till_none))
                    
        #print("undo_dict: ", undo_dict)
        print("redo: ", points_dict)
    
            


   


In [122]:
# Configuración de la cámara
cap = cv2.VideoCapture(0)

# Configuración de la pantalla para reescalar la captura de la cámara
screen_width, screen_height = pyautogui.size()

# Configurar el tamaño de la ventana de captura al tamaño de la pantalla
cap.set(cv2.CAP_PROP_FRAME_WIDTH, screen_width)  # Ancho de la ventana
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, screen_height)  # Alto de la ventana

# Bucle principal
while True:
    # Leer el frame de la cámara
    success, img = cap.read()
    if not success:
        break

    # Efecto espejo
    img = cv2.flip(img, 1)

    # Detectar manos
    hands, img = detector.findHands(img)

    # Dibujar las líneas entre puntos en la pantalla
    for (color, thickness, mode), points in points_dict.items():
        draw_paint_mode(img, points, mode, thickness, color)

    if hands:
        erase = update_actions()
        hand = hands[0]  # Solo la primera mano detectada
        lmList = hand["lmList"]  # Lista de posiciones (landmarks)
        fingers = detector.fingersUp(hand)  # Dedos levantados (1 levantado, 0 no levantado)

        # Coordenadas del dedo índice (punta)
        x_index, y_index = lmList[8][:2]  # Índice

        # Coordenadas del pulgar (punta)
        x_thumb, y_thumb = lmList[4][:2]  # Pulgar

        x = (x_index + x_thumb) // 2
        y = (y_index + y_thumb) // 2

        # Distancia entre el índice y el pulgar
        index_thumb_distance, _, _ = detector.findDistance(lmList[8][:2], lmList[4][:2], img)

        # Detectar si la mano está cerrada (puño) para borrar
        fingers_down = all(lmList[f][1] > lmList[f - 2][1] for f in [8, 12, 16, 20])
        erasing = False
        if detectar_gestos(hand, lmList) == "Puño cerrado":
            # Dibujar el círculo de borrado
            erasing = True
            cv2.circle(img, (x, y), current_thickness, (0, 255, 255), -1)  # Borde amarillo
            cv2.circle(img, (x, y), current_thickness - 2, (0, 0, 0), -1)  # Interior negro
            for (color, thickness, mode), points in points_dict.items():
                cleaned_points = []
                previous_was_none = False
                for point in points:
                    if point is None:
                        if not previous_was_none:
                            # Solo añadimos un None si el anterior no era None
                            cleaned_points.append(point)
                            previous_was_none = True
                    else:
                        # Si el punto está dentro del radio de borrado, lo eliminamos
                        if ((x - point[0])**2 + (y - point[1])**2)**0.5 <= current_thickness*2-2:
                            if not previous_was_none:
                                cleaned_points.append(None)  # Añadimos un None al borrar
                                previous_was_none = True
                        else:
                            cleaned_points.append(point)  # Mantén el punto
                            previous_was_none = False
                points_dict[(color, thickness, mode)] = cleaned_points
                
        elif detectar_gestos(hand, lmList) != "Sin gesto":
            # Activar interfaz de color, grosor o modo de pintura según posición del dedo
            if detectar_gestos(hand, lmList) == "Mano extendida":
                color_interface_active = True
                paint_mode_interface_active = False
                thickness_interface_active = False
                undo_mode_active = False
            elif detectar_gestos(hand, lmList) == "Llamada":
                color_interface_active = False
                paint_mode_interface_active = True
                thickness_interface_active = False
                undo_mode_active = False
            elif detectar_gestos(hand, lmList) == "Señal de paz":
                color_interface_active = False
                paint_mode_interface_active = False
                thickness_interface_active = True
                undo_mode_active = False
            elif detectar_gestos(hand, lmList) == "Dedo íncide arriba":
                color_interface_active = False
                paint_mode_interface_active = False
                thickness_interface_active = False
                undo_mode_active = True
                redo_mode_active = False
            elif detectar_gestos(hand, lmList) == "Tres dedos arriba":
                color_interface_active = False
                paint_mode_interface_active = False
                thickness_interface_active = False
                undo_mode_active = False
                redo_mode_active = False
                allow_undo = True
                allow_redo = True
            elif detectar_gestos(hand, lmList) == "L":
                cv2.putText(img, "L", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                color_interface_active = False
                paint_mode_interface_active = False
                thickness_interface_active = False
                undo_mode_active = False
                redo_mode_active = True
                
        # Solo permitir dibujar si ninguna interfaz está activa o si ha transcurrido el tiempo de delay en la selección de una opción
        elif not (color_interface_active or thickness_interface_active or paint_mode_interface_active) and has_delay_time_passed:
            if index_thumb_distance < 50:
                if (current_color, current_thickness, current_paint_mode) not in points_dict:
                    points_dict[(current_color, current_thickness, current_paint_mode)] = []
                if not drawing:
                    points_dict[(current_color, current_thickness, current_paint_mode)].append(None)
                drawing = True
                if len(points_dict[(current_color, current_thickness, current_paint_mode)]) > 0:
                    last_point = points_dict[(current_color, current_thickness, current_paint_mode)][-1]
                    if last_point is not None:
                        interpolated_points = interpolate_points(last_point, (x, y))
                        points_dict[(current_color, current_thickness, current_paint_mode)].extend(interpolated_points)
                    else:
                        points_dict[(current_color, current_thickness, current_paint_mode)].append((x, y))
            else:
                # Instante antes de dejar de dibujar para guardar el tipo de trazo
                if drawing == True:
                    # Cada vez que se termina de trazar una línea, se guarda el tipo de trazo en el historial
                    drawings_over_time.append((current_color, current_thickness, current_paint_mode))
                drawing = False


        # Interfaz de selección de colores
        if color_interface_active:
            # Dibujar la interfaz de selección de colores
            color_rectangles = draw_color_selection_interface(img)
            # Detectar si el clic está dentro de algún rectángulo
            current_color, color_interface_active = detect_color_selection(x, y, index_thumb_distance, color_rectangles, current_color, color_interface_active)
            
            if not color_interface_active:
                # Variable bool para controlar el delay time de la selección
                has_delay_time_passed = False
                selection_time = time.time()

        # Interfaz de selección de grosor
        if thickness_interface_active:
            # Dibujar la interfaz de selección de grosor
            thickness_circles = draw_thickness_selection_interface(img, current_color)

            if erase == "erasing":
                thickness_circles = draw_thickness_selection_interface(img, (0, 0, 0), (255, 255, 255))   
                         
            # Detectar si el clic está dentro de la selección de grosor
            current_thickness, thickness_interface_active = detect_thickness_selection(x, y, index_thumb_distance, thickness_circles, current_thickness, thickness_interface_active)
            
            if not thickness_interface_active:
                # Variable bool para controlar el delay time de la selección
                has_delay_time_passed = False
                selection_time = time.time()
            
        # Interfaz de selección de modo de pintura
        if paint_mode_interface_active:
            # Dibujar la interfaz de selección de modo de pintura
            mode_rectangles = draw_paint_mode_interface(img)

            # Detectar clics dentro de los rectángulos
            current_paint_mode, paint_mode_interface_active = detect_paint_mode_selection(x, y, index_thumb_distance, mode_rectangles, current_paint_mode, paint_mode_interface_active)

            if not paint_mode_interface_active:
                # Variable bool para controlar el delay time de la selección
                has_delay_time_passed = False
                selection_time = time.time()

        # Verificar si ha pasado el tiempo de retraso después de seleccionar alguna opción en una interfaz de selección
        if not has_delay_time_passed and (time.time() - selection_time > selection_delay):
            has_delay_time_passed = True

        if undo_mode_active and allow_undo:
            # Historial para rehacer cambios y historial de puntos deshechos
            undo()
            undo_mode_active = False
            allow_undo = False

        if redo_mode_active and allow_redo:
            if redo_history and undo_dict:
                redo(redo_history, undo_dict)
                redo_mode_active = False
                allow_redo = False

            
            
        # Condiciones para el gesto del pulgar hacia arriba
        if detect_thumbs_up(lmList, hands):
            if thumbs_up_start_time is None:
                thumbs_up_start_time = cv2.getTickCount()
            else:
                # Calcula el tiempo transcurrido en segundos
                elapsed_time = (cv2.getTickCount() - thumbs_up_start_time) / cv2.getTickFrequency()
                if elapsed_time >= time_to_capture:  # Verifica si han pasado al menos 1.5 segundos                   
                    # Crear una imagen en blanco (lienzo)
                    canvas = np.ones_like(img) * 255
                    
                    # Dibujar las líneas en el lienzo
                    for (color, thickness, mode), points in points_dict.items():
                        draw_paint_mode(canvas, points, mode, thickness, color)
                    prueba = canvas.copy()
                    cv2.floodFill(prueba, None, (0, 0), (200, 0, 0))          
                    # Guardar la imagen del lienzo con un número incremental
                    capture_number = 1
                    while True:
                        capture_filename = f"./capturas/captura_dibujo_{capture_number}.png"
                        prueba_filename = f"./capturas/fill_prueba_{capture_number}.png"
                        if not os.path.exists(capture_filename) and not os.path.exists(prueba_filename):
                            cv2.imwrite(capture_filename, canvas)
                            cv2.imwrite(prueba_filename, prueba)
                            break
                        capture_number += 1
                    
                    # Efecto de flash
                    flash_duration = 0.5  # Duración del flash en segundos
                    flash_start_time = cv2.getTickCount()
                    while (cv2.getTickCount() - flash_start_time) / cv2.getTickFrequency() < flash_duration:
                        flash_elapsed_time = (cv2.getTickCount() - flash_start_time) / cv2.getTickFrequency()
                        flash_img = img.copy()
                        # Reducción de transparencia durante el efecto de flash
                        alpha_flash = max(0, 0.5 * (1 - flash_elapsed_time / flash_duration))
                        # Aplicar el efecto de flash con transparencia
                        cv2.addWeighted(np.ones_like(flash_img) * 255, alpha_flash, flash_img, 1 - alpha_flash, 0, flash_img)
                        cv2.imshow("Dibujo", flash_img)
                        cv2.waitKey(1)
                    
                    # Reiniciar el temporizador después de la captura
                    thumbs_up_start_time = None
                
                else:
                    cv2.putText(img, "Capturando...", (50, 100), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 255, 0), 2)
                    # Ancho de la barra de progreso
                    progress = int((elapsed_time / time_to_capture) * 300) 
                    # Dibujar la barra de progreso
                    cv2.rectangle(img, (50, 115), (50 + progress, 170), (0, 255, 0), -1)
                    # Dibujar el cuadro de la barra de progreso
                    cv2.rectangle(img, (50, 115), (350, 170), (0, 255, 0), 2)
        else:
            thumbs_up_start_time = None  # Reiniciar si el gesto no está activo
            
    # Mostrar la imagen resultante
    cv2.imshow("Dibujo", img)

    # Salir con la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Points_dict sin mutar:  {((0, 0, 255), 10, 'normal'): [None, (865, 674), (864, 673), (863, 672), (862, 671), (861, 670), (860, 669), (860, 668), (859, 668), (858, 667), (857, 666), (856, 665), (855, 664), (855, 663), (854, 663), (853, 662), (853, 661), (852, 661), (851, 660), (850, 659), (850, 658), (849, 658), (848, 657), (848, 656), (847, 656), (846, 655), (846, 654), (845, 653), (844, 653), (843, 652), (843, 651), (842, 651), (841, 650), (841, 649), (840, 648), (839, 648), (839, 647), (838, 646), (837, 646), (836, 645), (836, 644), (835, 643), (834, 643), (834, 642), (833, 641), (832, 641), (832, 640), (831, 639), (830, 638), (829, 638), (829, 637), (828, 636), (827, 636), (827, 635), (826, 634), (825, 633), (824, 632), (823, 631), (822, 631), (822, 630), (821, 629), (820, 628), (819, 627), (818, 626), (817, 625), (816, 624), (815, 623), (814, 622), (813, 621), (812, 620), (811, 619), (811, 618), (810, 618), (809, 617), (808, 616), (807, 615), (806, 614), (806, 613), (805, 613), (80